In [ ]:
# Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import string
import json
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tkinter import messagebox
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# Crawler Component
URL = "https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/"
profile_url = "https://pureportal.coventry.ac.uk/en/persons/"

In [ ]:
def get_maximum_page():
    first = requests.get(URL)
    soup = BeautifulSoup(first.text, 'html.parser')
    final_page = soup.select('#main-content > div > section > nav > ul > li:nth-child(12) > a')[0]['href']
    fp = final_page.split('=')[-1]
    return int(fp)
mx = get_maximum_page()

In [ ]:
def check_department(researcher):
    
    l1 = researcher.find('div', class_='rendering_person_short')
      
    for span in l1.find_all('span'):
        # Check department
        if span.text == str('Centre for Intelligent Healthcare'):
            name = researcher.find('h3', class_='title').find('span').text
            
            return name
        else:
            pass

def remove_data_from_csv(file_path):
    # Read the CSV file and keep only the header row
    df = pd.read_csv(file_path, nrows=0)

    # Save the DataFrame with only the header row back to the CSV file
    df.to_csv(file_path, index=False)
    
def update_csv(database):
    current_data = pd.read_csv(database)
    return current_data        

def enter_each_researchers_publication(researcher, url, df):
    current_data = pd.read_csv(df)  # Load existing DataFrame from the CSV file

    new_url = url + str(researcher).replace(' ', '-').lower() + '/publications/'
    page = requests.get(new_url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="main-content")
    papers = results.find_all("li", class_="list-result-item")

    new_rows = []  # List to store the new rows

    for index, paper in enumerate(papers, start=len(current_data)+1):
        title_element = paper.find('h3', class_='title')
        title = title_element.find('span').text if title_element else None

        author_element = paper.find('a', class_='link person')
        author = author_element.find('span').text if author_element else None

        date_element = paper.find('span', class_="date")
        date = date_element.text if date_element else None

        link_element = paper.find('h3', class_='title').find('a', href=True)
        link = link_element['href'] if link_element else None

        new_row = {'SN': index,
                   'Title': title,
                   'Author': author,
                   'Published': date,
                   'Link': link}

        new_rows.append(new_row)  # Append the new row to the list

    new_data = pd.DataFrame(new_rows)  # Create a new DataFrame with the new rows
    updated_data = pd.concat([current_data, new_data], ignore_index=True)  # Concatenate the existing DataFrame with the new DataFrame

    updated_data.to_csv(df, index=False)


In [ ]:
def scrape(mx):
    df = r'database2.csv'
    i = 0
    while True:
        if i > 45:
            break

        if i > 0:
            url = URL + '?page=' + str(i)
        else:
            url = URL

        i = i + 1
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="main-content")
        researchers = results.find_all("li", class_="grid-result-item")

        for researcher in researchers:
            check = researcher.find('div', class_='stacked-trend-widget')
            if check:
                name = check_department(researcher)
                if name is None:
                    pass
                else:
                    enter_each_researchers_publication(name, profile_url, df)

In [ ]:
# create_csv()
remove_data_from_csv(r'database2.csv')
update_csv(database=r'database2.csv')
scrape(mx)

# Indexing Component
scraped_db = pd.read_csv('database2.csv')

In [ ]:

def preprocess_text(text):
    if isinstance(text, float):
        return ""
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # Tokenization
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join tokens back into a single string
    processed_text = " ".join(tokens)
    
    return processed_text

In [ ]:
def create_index():
    index = {}
    for i, row in scraped_db.iterrows():
        title = row['Title']
        author = row['Author']
        processed_title = preprocess_text(title)
        processed_author = preprocess_text(author)
        
        # Update index with title
        if processed_title not in index:
            index[processed_title] = []
        index[processed_title].append(i)
        
        # Update index with author
        if processed_author not in index:
            index[processed_author] = []
        index[processed_author].append(i)
    
    with open('index.json', 'w') as f:
        json.dump(index, f)


In [ ]:
create_index()

In [ ]:
import tkinter as tk
from tkinter import *
import numpy as np
from sklearn.metrics import confusion_matrix

# Load the index from the JSON file
with open('index.json', 'r') as f:
    index = json.load(f)

def search_publications():
    query = entry.get(1.0, "end")
    processed_query = preprocess_text(query)
    
    if processed_query in index:
        matching_indices = index[processed_query]
        matching_publications = scraped_db.loc[matching_indices]
        
        # True positives: Matching publications retrieved and relevant
        true_positives = len(matching_publications)
        
        # False negatives: Relevant publications not retrieved
        false_negatives = len(scraped_db) - true_positives
        
        # False positives: Non-relevant publications retrieved
        false_positives = 0
        
        # True negatives: Non-relevant publications not retrieved
        true_negatives = 0
        
        # Calculate false positives and true negatives
        for i, row in scraped_db.iterrows():
            if i not in matching_indices:
                false_positives += 1
            else:
                true_negatives += 1
        
        # Calculate the predicted and actual labels
        predicted_labels = [1] * true_positives + [0] * false_positives
        actual_labels = [1] * (true_positives + false_negatives) + [0] * (false_positives + true_negatives)
        
        # Adjust the length of the lists if they are different
        max_length = max(len(predicted_labels), len(actual_labels))
        predicted_labels += [0] * (max_length - len(predicted_labels))
        actual_labels += [0] * (max_length - len(actual_labels))
        
        # Calculate the confusion matrix
        cm = confusion_matrix(actual_labels, predicted_labels)
        
        # Display the confusion matrix
        messagebox.showinfo("Confusion Matrix", str(cm))
        
        # Display the matching publications
        messagebox.showinfo("Search Results", matching_publications.to_string())
    else:
        messagebox.showinfo("Search Results", "No matching publications found.")


In [ ]:
# Create the main window
from tkinter import *
import tkinter as tk
window = tk.Tk()
window.title("Coventry University-Publication Search")
window.geometry("600x650")
window.configure(bg="#919492")
window.resizable(0,0)
from PIL import ImageTk, Image
img=(Image.open("coventry-university-logo.png"))
imgg=img.resize((200,200))
imggg=ImageTk.PhotoImage(imgg)

lbl=Label(window,image=imggg,bg="#919492")
lbl.pack(side=TOP)
# Create and position the search label
label = tk.Label(window, text="Enter author name or title name:",fg="white",font=("Arial", 20,"italic"),bg="#919492")
label.pack(pady=10)

# Create and position the search entry
entry = tk.Text(window,height=10,width=30,font=("Arial", 20))
entry.pack()

# Create and position the search button
button = tk.Button(window, text="Search",font=("Arial", 20),bg="#10164B",fg="white",command=search_publications)
button.pack(pady=10)


# Start the main event loop
window.mainloop()